# RANS from DNS

Our goal here, in general, is to find new valid equations that describe fluid flow. 
We will try to find new closures for the steady RANS equations based on direct numerical
simulation (DNS) of a boundary layer.


## Steady RANS equations

$$
(\vec{U} \cdot \nabla) \vec{U}
+ \frac{1}{\rho} \nabla P 
- \nu \nabla^2 \vec{U}
= - \frac{1}{\rho} \nabla \cdot \mathbf{R},
$$

where in this case $\mathbf{R}$ is the Reynolds stress tensor.

In order to be rank-consistent, terms that "make up" 
$- \frac{1}{\rho} \nabla \cdot \mathbf{R}$ can be one of:

* Gradient of a scalar (e.g., pressure gradient)
* Laplacian of a vector (e.g., viscous stress)
* Inner product of a tensor with a vector
* A vector multiplied by a scalar (e.g., advection)

Some ideas for what the Reynolds stress residual term could be:

$$
- \frac{1}{\rho} \nabla \cdot \mathbf{R} = 
      a \nabla K 
    + b \left( \nabla \vec{U} \cdot \nabla P \right)
    + c \nabla \cdot \vec{U} \nabla P
$$

## Algorithm

1. Pick terms (in addition to non-Reynolds stress Navier--Stokes terms).
2. Create a random list of points in space that is at least as large as the number
   of terms.
3. At each point, acquire all data for all terms for all times.
4. Average data at each point for all times.
5. Solve for coefficients using a linear model.

In [ ]:
%load_ext autoreload
%autoreload 2

import seaborn
seaborn.set_theme()

## Using data from pyJHTDB

1. Pick a bunch of points randomly throughout the domain, at least more than the number of terms we want to test.
2. Add points in each direction for computing spatial derivatives.
3. Get $\vec{u}$, $p$, and their gradients for all time at all points in the list.
4. Calculate terms based on mean values.
5. Use a regression model to determine coefficients on each term.
6. Repeat this process to ensure the coefficients don't change?
7. Run a RANS simulation with this new model and check the results against the mean profiles.

In [ ]:
import pandas as pd

df = pd.read_hdf("data/jhtdb-transitional-bl/all-stats.h5", key="data")

In [ ]:
# Let's check continuity
div = df["dudx"] + df["dvdy"] + df["dwdz"]
div.describe()

The check above gives us an idea on how accurate these gradient calculations
are.

In [ ]:
df.columns

In [ ]:
ax = (
    df.loc[df.index.get_level_values("x")[-1000]]
    .reset_index()
    .plot(x="u", y="y", legend=False, ylabel="$U$", xlabel="$y$")
)

In [ ]:
# Check the momentum equation
from py_package import nu, rho

momx_no_res = (
    df.u * df.dudx
    + df.v * df.dudy
    + df.w * df.dudz
    + (1 / rho) * df.dpdx
    - nu * (df.d2udx2 + df.d2udy2 + df.d2udz2)
)

momx = momx_no_res + (df.duudx_fd + df.duvdy_fd)

momx.dropna().describe()

## Check the $y$-component of the momentum equation

$$
U \frac{\partial V}{\partial x}
+ V \frac{\partial V}{\partial y}
= - \frac{1}{\rho} \frac{\partial P}{\partial y}
+ \nu \left( 
    \frac{\partial^2 V}{\partial x^2}
    + \frac{\partial^2 V}{\partial y^2}
    \right)
$$

$$
- \left(
    \frac{\partial \overline{u'v'}}{\partial x}
    + \frac{\partial \overline{v'v'}}{\partial y}
    \right
)
$$

In [ ]:
# Check the y-component of the momentum equation
from py_package import nu, rho

momy_no_res = (
    df.u * df.dvdx
    + df.v * df.dvdy
    + (1 / rho) * df.dpdy
    - nu * (df.d2vdx2 + df.d2vdy2)
)

momy = momy_no_res + (df.duvdx_fd + df.duvdy_fd)

momy_no_res.dropna().describe()

In [ ]:
momy_no_res.plot.hist()

In [ ]:
# Let's see how large the Reynold's stress residual is at different locations
df1 = df.dropna().reset_index()
df1.plot.scatter(
    x="x", y="y", color=momy_no_res.dropna().values, cmap="viridis"
)

## Compute terms from averaged data

In [ ]:
# Let's check the time-averaged profiles
import pandas as pd
import h5py
import numpy as np


def read_profiles():
    """Read profile data from JHTDB HDF5 file, and assemble into a dictionary
    of NumPy arrays.
    """
    with h5py.File(
        "data/jhtdb-transitional-bl/time-ave-profiles.h5", "r"
    ) as f:
        data = {}
        for k in f.keys():
            kn = k.split("_")[0]
            if kn.endswith("m"):
                kn = kn[:-1]
            data[kn] = f[k][()]
    # Calculate some finite difference gradients
    dx = np.gradient(data["x"])
    dy = np.reshape(np.gradient(data["y"]), (224, 1))
    # dz = np.gradient(data["z"])
    # Correct fluctuation terms according to README
    # >uum is the time-averaged of u*u (not u'*u', where u'=u-um).
    # >So time-averaged of u'*u'=uum-um*um. Same for other quantities.
    for dim1 in ("u", "v", "w"):
        for dim2 in ("u", "v", "w"):
            if f"{dim1}{dim2}" in data:
                data[f"{dim1}{dim2}"] = (
                    data[f"{dim1}{dim2}"] - data[dim1] * data[dim2]
                )
    # Calculate gradients
    data["dpdx"] = np.gradient(data["p"], axis=1) / dx
    data["duudx"] = np.gradient(data["uu"], axis=1) / dx
    data["duvdx"] = np.gradient(data["uv"], axis=1) / dx
    data["duvdy"] = np.gradient(data["uv"], axis=0) / dy
    data["dvvdy"] = np.gradient(data["vv"], axis=0) / dy
    data["dudx"] = np.gradient(data["u"], axis=1) / dx
    data["dudy"] = np.gradient(data["u"], axis=0) / dy
    data["dvdx"] = np.gradient(data["v"], axis=1) / dx
    data["dvdy"] = np.gradient(data["v"], axis=0) / dy
    data["d2udx2"] = np.gradient(data["dudx"], axis=1) / dx
    data["d2udy2"] = np.gradient(data["dudy"], axis=0) / dy
    data["d2vdx2"] = np.gradient(data["dvdx"], axis=1) / dx
    data["d2vdy2"] = np.gradient(data["dvdy"], axis=0) / dy
    data["dpdx"] = np.gradient(data["p"], axis=1) / dx
    data["dpdy"] = np.gradient(data["p"], axis=0) / dy
    # data["dwdz"] = np.gradient(data["w"], axis=1) / dz
    return data


data = read_profiles()

## Plot RANS simulation versus averaged DNS data

In [ ]:
import plotly.express as px
import plotly
import glob
from IPython.display import display, HTML

# Workaround to show LaTeX labels
plotly.offline.init_notebook_mode()
display(
    HTML(
        '<script type="text/javascript" async '
        'src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/'
        '2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
    )
)

fig = px.line(x=data["u"][:, 1500], y=data["y"], height=800).update_traces(
    line_color="green", line_dash="dash", name="DNS"
)

turbulence_model = "k-epsilon"

for ny in [15, 20, 30, 40, 60]:
    fpaths = glob.glob(
        f"sim/cases/{turbulence_model}-ny-{ny}/"
        "postProcessing/sample/*/x906.8_U.csv"
    )
    assert len(fpaths) == 1
    df2 = pd.read_csv(fpaths[0])
    fig2 = df2.plot(x="U_0", y="y", backend="plotly").update_traces(
        name=f"$N_y = {ny}$"
    )
    fig = fig.add_trace(fig2.data[0])
    fig.data[0].name = "DNS"
    fig.data[1].name = r"$k$--$\epsilon$"
fig = fig.update_layout(
    showlegend=True, xaxis=dict(title=r"$U$"), yaxis=dict(title="$y$")
)
fig.show()

In [ ]:
import plotly.express as px
import plotly
import glob
from IPython.display import display, HTML

# Workaround to show LaTeX labels
plotly.offline.init_notebook_mode()
display(
    HTML(
        '<script type="text/javascript" async '
        'src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/'
        '2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
    )
)

fig = px.line(x=data["u"][:, 1500], y=data["y"], height=800).update_traces(
    line_color="green", line_dash="dash", name="DNS"
)

turbulence_model = "laminar"

for ny in [40]:
    fpaths = glob.glob(
        f"sim/cases/{turbulence_model}-ny-{ny}/"
        "postProcessing/sample/*/x906.8_U.csv"
    )
    assert len(fpaths) == 1
    df2 = pd.read_csv(fpaths[0])
    fig2 = df2.plot(x="U_0", y="y", backend="plotly").update_traces(
        name=f"$N_y = {ny}$"
    )
    fig = fig.add_trace(fig2.data[0])
    fig.data[0].name = "DNS"
    fig.data[1].name = r"$k$--$\epsilon$"
fig = fig.update_layout(
    showlegend=True, xaxis=dict(title=r"$U$"), yaxis=dict(title="$y$")
)
fig.show()

In [ ]:
import plotly.express as px
import plotly
import glob
from IPython.display import display, HTML

# Workaround to show LaTeX labels
plotly.offline.init_notebook_mode()
display(
    HTML(
        '<script type="text/javascript" async '
        'src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/'
        '2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
    )
)

fig = px.line(x=data["u"][:, 1500], y=data["y"], height=800).update_traces(
    line_color="green", line_dash="dash", name="DNS"
)

turbulence_model = "new"

for ny in [40]:
    fpaths = glob.glob(
        f"sim/cases/{turbulence_model}-ny-{ny}/"
        "postProcessing/sample/*/x906.8_U.csv"
    )
    assert len(fpaths) == 1
    df2 = pd.read_csv(fpaths[0])
    fig2 = df2.plot(x="U_0", y="y", backend="plotly").update_traces(
        name=f"$N_y = {ny}$"
    )
    fig = fig.add_trace(fig2.data[0])
    fig.data[0].name = "DNS"
    fig.data[1].name = r"$k$--$\epsilon$"
fig = fig.update_layout(
    showlegend=True, xaxis=dict(title=r"$U$"), yaxis=dict(title="$y$")
)
fig.show()

In [ ]:
import plotly.express as px

fig = (
    px.line(x=-nu * data["d2udy2"][:, 1500], y=data["y"], height=800)
    .update_traces(line_color="green", line_dash="dash", name="DNS")
    .show()
)

In [ ]:
import plotly.express as px

fig = (
    px.line(x=1 / rho * data["dpdx"][:, 1500], y=data["y"], height=800)
    .update_traces(line_color="green", line_dash="dash", name="DNS")
    .show()
)

In [ ]:
import plotly.express as px

fig = (
    px.line(x=data["dudy"][:, 1500], y=data["y"], height=800)
    .update_traces(line_color="green", line_dash="dash", name="DNS")
    .show()
)

In [ ]:
import plotly.express as px

fig = (
    px.line(x=data["dudy"][:, 1500]**(3)*data["y"], y=data["y"], height=800)
    .update_traces(line_color="green", line_dash="dash", name="DNS")
    .show()
)

## Detect coefficients of new terms

In [ ]:
# Compute a bunch of quantities and add to the data
data = read_profiles()
dx = np.gradient(data["x"])
dy = np.reshape(np.gradient(data["y"]), (224, 1))
# Advection of momentum
data["advection_x"] = data["u"] * data["dudx"] + data["v"] * data["dudy"]
data["advection_y"] = data["u"] * data["dvdx"] + data["v"] * data["dvdy"]
# Mean kinetic energy
data["k"] = 0.5 * (data["u"] ** 2 + data["v"] ** 2)
# Squared gradients
data["dudx2"] = data["dudx"] ** 2
data["dudy2"] = data["dudy"] ** 2
# Gradients of squares
data["du2dy"] = np.gradient(data["u"] ** 2, axis=0) / dy
data["dv2dy"] = np.gradient(data["v"] ** 2, axis=0) / dy
data["du2dx"] = np.gradient(data["u"] ** 2, axis=1) / dx
data["dv2dx"] = np.gradient(data["v"] ** 2, axis=1) / dx
# Mean kinetic energy gradient
data["dkdy"] = np.gradient(data["k"], axis=0) / dy
data["dkdx"] = np.gradient(data["k"], axis=1) / dx
data["d2kdy2"] = np.gradient(data["dkdy"], axis=0) / dy
data["d2kdx2"] = np.gradient(data["dkdx"], axis=1) / dx
# Gradients multiplied by each other
# Gradients multiplied by mean values
# Misc terms
data["u2dudy"] = data["u"] ** 2 * data["dudy"]
data["udpdx"] = data["u"] * data["dpdx"]
data["dpdx2"] = data["dpdx"] ** 2
data["dpdy2"] = data["dpdy"] ** 2
data["d2pdx2"] = np.gradient(data["dpdx"], axis=1) / dx
data["d2pdy2"] = np.gradient(data["dpdy"], axis=0) / dy
data["dp2dx"] = np.gradient(data["p"] ** 2, axis=1) / dx
data["dp2dy"] = np.gradient(data["p"] ** 2, axis=0) / dy
data["vdpdy"] = data["v"] * data["dpdy"]
data["duudy"] = np.gradient(data["uu"], axis=0) / dy
data["magsqrgradp"] = data["dpdx"] ** 2 + data["dpdy"] ** 2
data["laplacianp"] = data["d2pdx2"] + data["d2pdy2"]
data["laplacianvel_x"] = data["d2udx2"] + data["d2udy2"]
data["laplacianvel_y"] = data["d2vdx2"] + data["d2vdy2"]
data["vorticityz"] = data["dvdx"] - data["dudy"]
data["gradmagvort_x"] = np.gradient(data["vorticityz"], axis=1) / dx
data["gradmagvort_y"] = np.gradient(data["vorticityz"], axis=0) / dy
data["pu"] = data["p"] * data["u"]
data["pv"] = data["p"] * data["v"]
data["divvelpgrad_x"] = data["u"] * data["d2pdx2"] + data["u"] * data["d2pdy2"]
data["divvelpgrad_y"] = data["v"] * data["d2pdx2"] + data["v"] * data["d2pdy2"]
data["duvdy"] = np.gradient(data["uv"], axis=0) / dy
data["duvdx"] = np.gradient(data["uv"], axis=1) / dx
data["divuu_x"] = data["du2dx"] + data["duvdy"]
data["divuu_y"] = data["duvdx"] + data["dv2dy"]
data["magsqru"] = data["u"] ** 2 + data["v"] ** 2
data["pbymagsqru"] = data["p"] / data["magsqru"]
data["gradpbymagsqru_x"] = np.gradient(data["pbymagsqru"], axis=1) / dx
data["gradpbymagsqru_y"] = np.gradient(data["pbymagsqru"], axis=0) / dy
data["divp"] = data["dpdx"] + data["dpdy"]
data["ddivpdx"] = np.gradient(data["divp"], axis=1) / dx
data["ddivpdy"] = np.gradient(data["divp"], axis=0) / dy
data["gradugradpx"] = data["dudx"] * data["dpdx"] + data["dudy"] * data["dpdy"]
data["gradugradpy"] = data["dvdx"] * data["dpdx"] + data["dvdy"] * data["dpdy"]
# Reynolds stress terms
data["restress_x"] = data["duudx"] + data["duvdy"]
data["restress_y"] = data["duvdx"] + data["dvvdy"]

In [ ]:
from py_package.plotting import plot_heatmap

plot_heatmap(
    data["uu"],
    data,
    interactive=False,
    ix_min=600,
    iy_max=100,
)

In [ ]:
from py_package.plotting import plot_heatmap

plot_heatmap(
    data["uv"],
    data,
    interactive=False,
    ix_min=600,
    iy_max=100,
)

In [ ]:
from py_package.plotting import plot_heatmap

plot_heatmap(
    data["vv"],
    data,
    interactive=False,
    ix_min=600,
    iy_max=100,
)

In [ ]:
plot_heatmap(
    data["restress_y"],
    data,
    interactive=False,
    ix_min=600,
    iy_max=100,
)

In [ ]:
plot_heatmap(
    data["u"] * data["dpdy"],
    data,
    interactive=False,
    ix_min=600,
    iy_max=100,
)

In [ ]:
plot_heatmap(
    data["duvdy"],
    data,
    interactive=False,
    ix_min=600,
    iy_max=100,
)

In [ ]:
plot_heatmap(
    -(data["dkdx"] + data["dkdy"])**2,
    data,
    interactive=False,
    ix_min=600,
    iy_max=100,
)

In [ ]:
from sklearn.linear_model import LinearRegression

terms = [
    # Mean kinetic energy gradients
    "dkdx",
    # "dkdy",
    # Squared velocity gradients
    # "dudx2",
    # "dudy2",
    # Gradients of squared velocity
    # "du2dx",
    # "du2dy",
    # "dv2dx",
    # "dv2dy",
    # Cross-stream pressure gradient
    # "dpdy",
    # Misc
    # "u2dudy",
    # "udpdx",
    # "dpdx2",
    "dpdy2",
    "d2pdx2",
    "d2pdy2",
    # "dp2dx",
    # "dpdx",
]

# Form our X matrix
X = np.array([data[term].flatten() for term in terms]).transpose()

# Form our y vector, which is simply the x-component of the 2-D RANS equations
# with no Reynolds stresses
y = (
    data["u"] * data["dudx"]
    + data["v"] * data["dudy"]
    + 1 / rho * data["dpdx"]
    - nu * (data["d2udx2"] + data["d2udy2"])
).flatten()

print("Dataset size:", len(y))

reg = LinearRegression(fit_intercept=False).fit(X, y)
print("Score:", reg.score(X, y))
print()
for term, coeff in zip(terms, reg.coef_):
    print(term, coeff)

In [ ]:
from sklearn.linear_model import LinearRegression

terms = [
    # Mean kinetic energy gradients
    # "dkdx",
    "dkdy",
    # Squared velocity gradients
    # "dudx2",
    # "dudy2",
    # Gradients of squared velocity
    # "du2dx",
    # "du2dy",
    # "dv2dx",
    # "dv2dy",
    # Streamwise pressure gradient
    # "dpdx",
    # Misc
    # "u2dudy",
    # "udpdx",
    "dpdx2",
    "dpdy2",
    # "d2pdx2",
    "d2pdx2",
    "d2pdy2",
    # "dp2dx",
    # "dpdy",
]

# Form our X matrix
X = np.array([data[term].flatten() for term in terms]).transpose()

# Form our y vector, which is simply the x-component of the 2-D RANS equations
# with no Reynolds stresses
y = (
    data["u"] * data["dvdx"]
    + data["v"] * data["dvdy"]
    + 1 / rho * data["dpdy"]
    - nu * (data["d2vdx2"] + data["d2vdy2"])
).flatten()
print("Dataset size:", len(y))

reg = LinearRegression(fit_intercept=False).fit(X, y)
print("Score:", reg.score(X, y))
print()
for term, coeff in zip(terms, reg.coef_):
    print(term, coeff)

In [ ]:
from sklearn.linear_model import LinearRegression

# Now we need to solve for both components at the same time so we use the same
# parameters for each
terms = [
    # "dkd{dim}",
    "gradugradp{dim}",
    # "restress_{dim}",  # Should be the most important term, but cheating
    "divuu_{dim}",
    "divvelpgrad_{dim}",
    "ddivpd{dim}",
    "gradpbymagsqru_{dim}",
    # "advection_{dim}",
    # "dpd{dim}",  # Cheating?
    # "laplacianvel_{dim}",  # Cheating?
]
xterms = [t.format(dim="x") for t in terms]
yterms = [t.format(dim="y") for t in terms]

# Slice out "non-important" parts of the flow to not train on those
# Basically that means stay near the wall and away from the inlet
ix_min = 500
iy_max = 100

# Form our X matrix
xx = np.array([data[term].flatten() for term in xterms]).transpose()
xy = np.array([data[term].flatten() for term in yterms]).transpose()
X = np.concatenate([xx, xy])
xx_slice = np.array(
    [data[term][:iy_max, ix_min:].flatten() for term in xterms]
).transpose()
xy_slice = np.array(
    [data[term][:iy_max, ix_min:].flatten() for term in yterms]
).transpose()
X_slice = np.concatenate([xx_slice, xy_slice])

xtarget = (
    data["u"] * data["dudx"]
    + data["v"] * data["dudy"]
    + 1 / rho * data["dpdx"]
    - nu * (data["d2udx2"] + data["d2udy2"])
)
ytarget = (
    data["u"] * data["dvdx"]
    + data["v"] * data["dvdy"]
    + 1 / rho * data["dpdy"]
    - nu * (data["d2vdx2"] + data["d2vdy2"])
)
xtarget = data["restress_x"]
ytarget = data["restress_y"]
# xtarget = np.zeros(data["u"].shape)
# ytarget = np.zeros(data["v"].shape)

# Our y vector is both of these appended to each other
y = np.concatenate([xtarget.flatten(), ytarget.flatten()])
y_slice = np.concatenate(
    [xtarget[:iy_max, ix_min:].flatten(), ytarget[:iy_max, ix_min:].flatten()]
)
print("Dataset size:", len(y_slice))

# Fit the model
reg = LinearRegression(fit_intercept=False).fit(X_slice, y_slice)

# Compute the RMSE of the fit
rmse_no_model = np.sqrt(np.sum(y**2))
pred = reg.predict(X)
error = y - pred
rmse_model = np.sqrt(np.sum(error**2))
error_x = error[: len(error) // 2].reshape((len(data["y"]), len(data["x"])))
error_y = error[len(error) // 2 :].reshape((len(data["y"]), len(data["x"])))

print("Score:", reg.score(X, y))
print("RMSE no model:", rmse_no_model)
print("RMSE model:", rmse_model)
print()
for term, coeff in zip(terms, reg.coef_):
    print(term, coeff)

### The null space approach

In [ ]:
terms = [
    "dkd{dim}",
    # "gradugradp{dim}",
    # "restress_{dim}",
    # "divuu_{dim}",
    # "divvelpgrad_{dim}",
    "ddivpd{dim}",
    "gradpbymagsqru_{dim}",
    "advection_{dim}",
    "dpd{dim}",
    "laplacianvel_{dim}",
]

# The RHS of our system of equations?
# J =
xi_min = 600
yi_max = 100


def nullspace(A, eps=1e-15):
    u, s, vh = np.linalg.svd(A)
    null_space = np.compress(s <= eps, vh, axis=0)
    return null_space.T

iterations = 1000

all_coeffs = []

for _ in range(iterations):
    K = np.zeros((len(terms), len(terms)))
    for n in range(len(terms)):
        # Pick random indices
        xi = np.random.choice(np.arange(len(data["x"]))[xi_min:])
        yi = np.random.choice(np.arange(len(data["y"]))[:yi_max])
        for m, term in enumerate(terms):
            K[n, m] = (
                data[term.format(dim="x")][yi, xi]
                - data[term.format(dim="y")][yi, xi]
            )
    try:
        M = nullspace(K, eps=1e-4)
        coeffs = (M.T/M[0])[0]
        # coeffs /= np.max(np.abs(coeffs))
        all_coeffs.append(coeffs)
    except ValueError:
        pass

all_coeffs = np.asarray(all_coeffs)
df = pd.DataFrame(all_coeffs, columns=terms)
dfm = df.mean()
dfm / dfm.abs().max()

In [ ]:
df.describe()

In [ ]:
# Let's plot the distribution of errors coming from the model
import py_package.plotting
from py_package.plotting import plot_heatmap

plot_heatmap(
    error_x,
    data,
    ix_min=600,
    iy_max=100,
    diverging=True,
    interactive=False,
)

In [ ]:
plot_heatmap(
    data["dkdx"],
    data,
    ix_min=600,
    iy_max=100,
    diverging=True,
    interactive=False,
)

In [ ]:
plot_heatmap(
    data["duvdy"],
    data,
    ix_min=600,
    iy_max=100,
    diverging=False,
    interactive=False,
)

## Attempting to model the (1, 2) component of the Reynolds stress transport

From the inspections above, the cross-stream gradient of 
$\overline{u^\prime v^\prime}$ is the largest driver of momentum transport
in the streamwise direction.
Therefore, we should try to find a PDE that describes this field well,
such that we can solve that and plug it back into the momentum
equation.

We could use something like

$$
\left( 
    \nabla \cdot \vec{U}
\right) \mathbf{R}
=
  a \nabla^2 \mathbf{R}
+ b \vec{U} \nabla P
+ c \nabla \vec{U}
+ d \vec{U} \nabla K
+ e \nabla P \nabla P
$$

for which the (1, 2) component is

$$
  U \frac{\partial \overline{u^\prime v^\prime}}{\partial x}
+ V \frac{\partial \overline{u^\prime v^\prime}}{\partial y}
=
  a \left( 
    \frac{\partial^2 \overline{u^\prime v^\prime}}{\partial x^2} 
    + \frac{\partial^2 \overline{u^\prime v^\prime}}{\partial y^2} 
  \right)
+ b U \frac{\partial P}{\partial y}
+ c \frac{\partial U}{\partial y}
+ d U \frac{\partial K}{\partial y}
+ e \frac{\partial P}{\partial x} \frac{\partial P}{\partial y}
$$

In [ ]:
# Compute a bunch of quantities and add to the data
data = read_profiles()
dx = np.gradient(data["x"])
dy = np.reshape(np.gradient(data["y"]), (224, 1))
data["k"] = 0.5 * (data["u"] ** 2 + data["v"] ** 2)
data["dkdy"] = np.gradient(data["k"], axis=0) / dy
data["dpdx"] = np.gradient(data["p"], axis=1) / dx
data["dpdy"] = np.gradient(data["p"], axis=0) / dy
data["duvdx"] = np.gradient(data["uv"], axis=1) / dx
data["duvdy"] = np.gradient(data["uv"], axis=0) / dy
data["d2uvdx2"] = np.gradient(data["duvdx"], axis=1) / dx
data["d2uvdy2"] = np.gradient(data["duvdy"], axis=0) / dy
data["adv_uv_12"] = data["u"] * data["duvdx"] + data["v"] * data["duvdy"]
data["laplacian_uv_12"] = data["d2uvdx2"] + data["d2uvdy2"]
data["u_gradp_12"] = data["u"] * data["dpdy"]
data["u_gradk_12"] = data["u"] * data["dkdy"]
data["gradp_gradp_12"] = data["dpdx"] * data["dpdy"]

In [ ]:
from sklearn.linear_model import LinearRegression

terms = [
    "laplacian_uv_12",
    "u_gradp_12",
    "dudy",
    "u_gradk_12",
    # "gradp_gradp_12",
]
# Form our X matrix
X = np.array([data[term].flatten() for term in terms]).transpose()
# Form our y vector, which is the advection term for uv
y = data["adv_uv_12"].flatten()
# Fit the model
reg = LinearRegression(fit_intercept=False).fit(X, y)
# Compute the RMSE of the fit
rmse_no_model = np.sqrt(np.sum(y**2))
pred = reg.predict(X)
error = y - pred
rmse_model = np.sqrt(np.sum(error**2))
error = error.reshape((len(data["y"]), len(data["x"])))
# Print metrics
print("Score:", reg.score(X, y))
print("RMSE no model:", rmse_no_model)
print("RMSE model:", rmse_model)
print()
for term, coeff in zip(terms, reg.coef_):
    print(term, coeff)

In [ ]:
plot_heatmap(
    data["duvdy"],
    data,
    ix_min=600,
    iy_max=100,
    diverging=True,
    interactive=False,
)

In [ ]:
plot_heatmap(
    data["gradp_gradp_12"],
    data,
    ix_min=600,
    iy_max=100,
    diverging=True,
    interactive=False,
)

In [ ]:
plot_heatmap(
    data["u_gradk_12"],
    data,
    ix_min=600,
    iy_max=100,
    diverging=False,
    interactive=False,
)

In [ ]:
plot_heatmap(
    data["dudy"],
    data,
    ix_min=600,
    iy_max=100,
    diverging=False,
    interactive=False,
)